In [59]:
# Обеспечим совместимость с Python 2 и 3
# pip install future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# импортируем Pandas и Numpy
import pandas as pd
import numpy as np
df = pd.read_csv('X_train.csv')
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 1000)
cols = list(df.columns.values)
print(cols)

df_nes=df[['sku', 'categoryLevel2Id' , 'brandId', 'userName', 'reting' , 'comment' , 'commentNegative', 'commentPositive']]

#df_nes.head(n=100)
df_nes_learn=df_nes.head(n=10000)
df_nes_test=df_nes.tail(n=5550)
#df_nes_test.head()
#df_nes_learn.head()

['sku', 'categoryLevel1Id', 'categoryLevel2Id', 'brandId', 'property', 'userName', 'reting', 'date', 'comment', 'commentNegative', 'commentPositive']


In [55]:
df_nes_learn.head()

,sku,categoryLevel2Id,brandId,userName,reting,comment,commentNegative,commentPositive
0,20005023,4010201,826,b2898a81b45310b30beb8fc0c0a9ce1e,2.0,"2,5 года работала и все...устала! Лампочка гор...",NaN,NaN
1,20020647,4030101,1425,538c73d64461e13907bb95c51c38bfbc,2.0,Через 2 месяца после истечении гарантийного ср...,NaN,NaN
2,20020701,4010401,124,ddca2d0101513a6209db7868eed8be05,4.0,пользуюсь уже три недели. нареканий ни каких н...,NaN,NaN
3,30012256,2030301,93,289c20015b3713a82ba5ddf774d996f7,5.0,Ребят этот системный блок подойдёт для игры кс...,NaN,NaN
4,30011341,2050201,656,5576f82d149d4f688644fef2322c63ef,5.0,"я считаю, что яри замечательный телефон! Прият...",NaN,NaN


In [56]:
df_nes_test.head()

,sku,categoryLevel2Id,brandId,userName,reting,comment,commentNegative,commentPositive
10037,20023434,4050301,64,33b67031842d4c56d9dc03571081b77d,5.0,"Покупкой доволен,прежде чем купить изучал инос...",NaN,NaN
10038,20021798,4110104,10,fc7d8d59ae565bbc0d43d6a24d0f888f,5.0,Пылесос просто супер. очень довольны!,NaN,NaN
10039,20021697,4110102,80,6421219d872c945f8aa9dc434c923394,5.0,Пользуюсь месяц и не вижу ни каких проблем. Мо...,NaN,NaN
10040,11023744,1070908,13,3d801da09e7d82668e226799d9db91dc,2.0,Купил я этот телевизор Маме на день рождение. ...,NaN,NaN
10041,40054798,3030501,1542,560e1a279bae50a0248e626224828b4f,5.0,Лучшего симулятора я ещё не встречал. Хорошая ...,NaN,NaN


In [75]:
from distutils.core import setup, Extension
import os.path
import sys

try:
    from Cython.Distutils import build_ext
    have_pyrex = 1
except:
    have_pyrex = 0

# Directory which libstemmer sources are unpacked in.
library_dir = 'libstemmer_c'

if 'bootstrap' in sys.argv:
    from tarballfetcher import download_and_extract_tarball
    download_and_extract_tarball(
        'http://snowball.tartarus.org/dist/libstemmer_c.tgz',
        expected_md5='6f32f8f81cd6fa0150333ab540af5e27')
    sys.argv.remove('bootstrap')

if not os.path.exists(library_dir):
    sys.stderr.write(
        'WARNING: Directory `%s` does not exist. ' % library_dir +
        'To download it, invoke setup.py with `bootstrap`.\n')

# Directories in libstemmer which contain libstemmer sources (ie, not
# examples, etc).
library_core_dirs = ('src_c', 'runtime', 'libstemmer', 'include')

# Read the manifest of files in libstemmer.
src_files = [os.path.join(library_dir, line.strip().replace(' \\', ''))
             for line in open(os.path.join(library_dir, 'mkinc_utf8.mak'))
             if len(line.strip()) > 2
             and (line.strip().endswith('.c \\') or line.strip().endswith('.c'))
             and os.path.split(line.strip().replace(' \\', ''))[0] in library_core_dirs]

# Set the include path to include libstemmer.
include_dirs = ('src', os.path.join(library_dir, 'include'))

if have_pyrex:
    # Add the pyrex sources, and a special rule so distutils knows how to
    # use them.
    src_files.append('src/Stemmer.pyx')
    cmdclass = {'build_ext': build_ext}
else:
    # Add just the C sources.
    src_files.append('src/Stemmer.c')
    cmdclass = {}
      
long_description = r"""

Stemming algorithms

PyStemmer provides access to efficient algorithms for calculating a
"stemmed" form of a word.  This is a form with most of the common
morphological endings removed; hopefully representing a common
linguistic base form.  This is most useful in building search engines
and information retrieval software; for example, a search with stemming
enabled should be able to find a document containing "cycling" given the
query "cycles".

PyStemmer provides algorithms for several (mainly european) languages,
by wrapping the libstemmer library from the Snowball project in a Python
module.

It also provides access to the classic Porter stemming algorithm for
english: although this has been superseded by an improved algorithm, the
original algorithm may be of interest to information retrieval
researchers wishing to reproduce results of earlier experiments.

""".strip()

version_str = '1.3.0'
setup(name = 'PyStemmer',
      version = version_str,
      author = 'Richard Boulton',
      author_email = 'richard@tartarus.org',
      maintainer = 'Richard Boulton',
      maintainer_email = 'richard@tartarus.org',
      url = 'http://snowball.tartarus.org/',
      download_url = 'http://snowball.tartarus.org/wrappers/PyStemmer-%s.tar.gz' % version_str,
      description = 'Snowball stemming algorithms, for information retrieval',
      long_description = long_description,
      platforms = ["any"],
      license = ["MIT", "BSD"],
      keywords = [
      "python",
      "information retrieval",
      "language processing",
      "morphological analysis",
      "stemming algorithms",
      "stemmers"
      ],
      classifiers = [
      "Development Status :: 5 - Production/Stable",
      "Intended Audience :: Developers",
      "License :: OSI Approved :: MIT License",
      "License :: OSI Approved :: BSD License",
      "Natural Language :: Danish",
      "Natural Language :: Dutch",
      "Natural Language :: English",
      "Natural Language :: Finnish",
      "Natural Language :: French",
      "Natural Language :: German",
      "Natural Language :: Italian",
      "Natural Language :: Norwegian",
      "Natural Language :: Portuguese",
      "Natural Language :: Russian",
      "Natural Language :: Spanish",
      "Natural Language :: Swedish",
      "Operating System :: OS Independent",
      "Programming Language :: C",
      "Programming Language :: Other",
      "Programming Language :: Python",
      "Programming Language :: Python :: 2",
      "Programming Language :: Python :: 2.6",
      "Programming Language :: Python :: 2.7",
      "Programming Language :: Python :: 3",
      "Programming Language :: Python :: 3.2",
      "Programming Language :: Python :: 3.3",
      "Topic :: Database",
      "Topic :: Internet :: WWW/HTTP :: Indexing/Search",
      "Topic :: Text Processing :: Indexing",
      "Topic :: Text Processing :: Linguistic",
      ],

      ext_modules = [Extension('Stemmer', src_files,
                               include_dirs = include_dirs)],
      cmdclass = cmdclass
     )


FileNotFoundError: [Errno 2] No such file or directory: 'libstemmer_c\\mkinc_utf8.mak'

Всё нужное импортировали, можно писать код